# Лабораторная работа на тему "Прямые методы решения слау"

### Вариант 10
Эскалаторный метод решения слау
(c 191)

##### описание метода


эскалаторный метод применяется для решения системы линейных алгебраических уравнений (СЛАУ) вида: $Ax=b$ \
где $A = A^T$ \
A - симметричная матрица n×n \
x - вектор неизвестных \
b - вектор правых частей

In [2]:
import numpy as np

In [3]:
def solve(A: np.ndarray, b: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    n = A.shape[0]
    Ab = np.concatenate((A, np.expand_dims(-b, axis=1)), axis=1)
    Ab = np.concatenate((Ab, np.array([np.append(-b, 0)])), axis=0)
    
    Z = np.eye(n + 1)
    C = np.zeros((n+1, n))
    G = np.zeros((n+1, n))
    
    for k in range(n):
        # k - элемент диагонали
        j = k

        for i in range(j, n+1):
            # C_ij = a_i1*z_1j + a_i2*z_2j + ... +  a_ij-1*z_j-1j + a_ij, i>=j
            C[i, j] = np.sum(Ab[i,:j] * Z[:j,j]) + Ab[i,j]

        for i in range(j, n+1):
            # y_ij = c_ij/c_jj
            G[i, j] = C[i, j] / C[j, j]

        for i in range(k, -1, -1):
            # formula 9
            Z[i, k+1] = -G[k+1,i] - np.sum(Z[i,(i+1):(k+1)] * G[k+1,(i+1):(k+1)])

    x = Z[:n, n]
    return x, Z, C, G



In [4]:
def check_input_matrix(A: np.ndarray, b: np.ndarray) -> bool:
    return np.all(A == A.T) and np.all(np.linalg.solve(A, b))

In [5]:
def test(A: np.ndarray, b: np.ndarray) -> None:
    print(f"исходная матрица A {A}")
    print(f"исходный вектор b {b}")
    print(f"проверка на валидность входных данных {check_input_matrix(A, b)}")

    my_x, Z, C, G = solve(A, b)
    np_x = np.linalg.solve(A, b)
    error = abs(my_x - np_x)

    print(
        f"мой x: {my_x}",
        f"точный x: {np_x}",
        f"Ошибка: {error}",
        f"Максимальная ошибка: {np.max(error):.6e}",
        sep="\n"
    )
    print(
        "-"*10,
        f"Z:{Z}",
        "-"*10,
        f"C:{C}",
        "-"*10,
        f"G:{G}",
        "-"*10,
        sep="\n"
    )

In [6]:
A = np.array([
    [2, 3],
    [3, 4]
])
b = np.array([5, 7])

In [7]:
test(A, b)

исходная матрица A [[2 3]
 [3 4]]
исходный вектор b [5 7]
проверка на валидность входных данных True
мой x: [1. 1.]
точный x: [1. 1.]
Ошибка: [1.77635684e-15 1.33226763e-15]
Максимальная ошибка: 1.776357e-15
----------
Z:[[ 1.  -1.5  1. ]
 [ 0.   1.   1. ]
 [ 0.   0.   1. ]]
----------
C:[[ 2.   0. ]
 [ 3.  -0.5]
 [-5.   0.5]]
----------
G:[[ 1.   0. ]
 [ 1.5  1. ]
 [-2.5 -1. ]]
----------


In [8]:
A = np.array([
    [12, 1.5, 2, 2.5, 1],
    [1.5, 10, 1.2, 1.8, 0.9],
    [2, 1.2, 14, 3, 1.5],
    [2.5, 1.8, 3, 16, 1.8],
    [1, 0.9, 1.5, 1.8, 15]
])
b = np.array([8.5, 6.3, 10.2, 12.8, 7.1])

In [9]:
test(A, b)

исходная матрица A [[12.   1.5  2.   2.5  1. ]
 [ 1.5 10.   1.2  1.8  0.9]
 [ 2.   1.2 14.   3.   1.5]
 [ 2.5  1.8  3.  16.   1.8]
 [ 1.   0.9  1.5  1.8 15. ]]
исходный вектор b [ 8.5  6.3 10.2 12.8  7.1]
проверка на валидность входных данных True
мой x: [0.43803631 0.37754672 0.47986729 0.56471356 0.30572575]
точный x: [0.43803631 0.37754672 0.47986729 0.56471356 0.30572575]
Ошибка: [1.11022302e-16 5.55111512e-17 5.55111512e-17 1.11022302e-16
 0.00000000e+00]
Максимальная ошибка: 1.110223e-16
----------
Z:[[ 1.         -0.125      -0.15456476 -0.16160959 -0.04551911  0.43803631]
 [ 0.          1.         -0.09681529 -0.13419502 -0.0587018   0.37754672]
 [ 0.          0.          1.         -0.1796962  -0.07755673  0.47986729]
 [ 0.          0.          0.          1.         -0.0842418   0.56471356]
 [ 0.          0.          0.          0.          1.          0.30572575]
 [ 0.          0.          0.          0.          0.          1.        ]]
----------
C:[[12.          0.       

In [10]:
def generate(size: int = 4, tryings: int = 10**7) -> tuple[np.ndarray, np.ndarray]:
    for _ in range(tryings):
        # A_random = np.random.randn(size, size)
        A_random = np.random.randint(-10, 10, size=(size, size))
        A = (A_random + A_random.T)

        b = np.random.randint(-10, 10, size=(size))
        if check_input_matrix(A, b):
            return A, b

In [11]:
A, b = generate(size=4)
A, b

(array([[  8,   2,  -7,   7],
        [  2, -12,  -6,   6],
        [ -7,  -6, -16,  -4],
        [  7,   6,  -4, -10]]),
 array([ 7, -5,  5,  0]))

In [12]:
test(A, b)

исходная матрица A [[  8   2  -7   7]
 [  2 -12  -6   6]
 [ -7  -6 -16  -4]
 [  7   6  -4 -10]]
исходный вектор b [ 7 -5  5  0]
проверка на валидность входных данных True
мой x: [-0.38836105  0.93962787 -0.63103721  0.54433888]
точный x: [-0.38836105  0.93962787 -0.63103721  0.54433888]
Ошибка: [2.77555756e-16 1.11022302e-16 1.11022302e-16 1.11022302e-16]
Максимальная ошибка: 2.775558e-16
----------
Z:[[ 1.         -0.25        0.96       -0.92843327 -0.38836105]
 [ 0.          1.         -0.34        0.32882012  0.93962787]
 [ 0.          0.          1.          0.03288201 -0.63103721]
 [ 0.          0.          0.          1.          0.54433888]
 [ 0.          0.          0.          0.          1.        ]]
----------
C:[[  8.           0.           0.           0.        ]
 [  2.         -12.5          0.           0.        ]
 [ -7.          -4.25       -20.68         0.        ]
 [  7.           4.25         0.68       -14.65764023]
 [ -7.           6.75       -13.42         7.9